In [ ]:
!pip install eli5
!pip install shap
!pip install pdpbox

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #for plotting

pd.options.mode.chained_assignment = None  #hide any pandas warnings

In [ ]:
# This allows us to drag a csv file to open
# Only for google Colab
from google.colab import files
uploaded = files.upload()
dt = pd.read_csv("./heart_disease.csv")

In [ ]:
dt.head(10)

In [ ]:
dt.head()

In [ ]:
# Check data types
dt.dtypes

In [ ]:
#Fix data types => Turn certain fields into strings
dt['sex'] = dt['sex'].astype('object')
dt['cp'] = dt['cp'].astype('object')
dt['fbs'] = dt['fbs'].astype('object')
dt['restecg'] = dt['restecg'].astype('object')
dt['exang'] = dt['exang'].astype('object')
dt['slope'] = dt['slope'].astype('object')
dt['thal'] = dt['thal'].astype('object')

In [ ]:
dt.dtypes

In [ ]:
# Turn the objects with 'object' type into separate columns with their values
dt = pd.get_dummies(dt, drop_first=True)

In [ ]:
dt.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.model_selection import train_test_split #for data splitting

X_train, X_test, y_train, y_test = train_test_split(dt.drop('target', 1), dt['target'], test_size = .2, random_state=10) #split the data

In [ ]:
# Random forest and set max depth, this also defaults to decision tree classifier to create the splits.
model = RandomForestClassifier(max_depth=5)
model.fit(X_train, y_train)

In [ ]:
estimator = model.estimators_[1]

# Get our feature names
feature_names = [i for i in X_train.columns]

#Properly label y_training
y_train_str = y_train.astype('str')
y_train_str[y_train_str == '0'] = 'no disease'
y_train_str[y_train_str == '1'] = 'disease'
y_train_str = y_train_str.values

In [ ]:
# Graph code stolen from https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c
# Export using graphviz
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_names,
                class_names = y_train_str,
                rounded = True, proportion = True, 
                label='root',
                precision = 2, filled = True)

# This is actually a bash command
!dot -Tpng tree.dot -o tree_out.png -Gdpi=600

# Display the image from the iPython shell
from IPython.display import Image
Image(filename = 'tree_out.png', width=1000,height=400)

In [ ]:
# Create predictions with model.predict
y_pred_bin = model.predict(X_test)

In [ ]:
# Get accuracy and confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix #for model evaluation

accuracy = accuracy_score(y_test, y_pred_bin)
cmatrix = confusion_matrix(y_test, y_pred_bin)
print("Accuracy: \n", accuracy)
print("Cmatrix: \n", cmatrix)

In [ ]:
# Basic sensitivity analysis
total=sum(sum(cmatrix))

sensitivity = cmatrix[0,0]/(cmatrix[0,0]+cmatrix[1,0])
print('Sensitivity : ', sensitivity )

specificity = cmatrix[1,1]/(cmatrix[1,1]+cmatrix[0,1])
print('Specificity : ', specificity)

In [ ]:
# Get permutation importance, how each feature affects the model
# Note this changes with different subsets of the data, depending on X_test
import eli5 #for purmutation importance
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
import shap #for SHAP values
from pdpbox import pdp, info_plots #for partial plots
# Plot PDP
base_features = dt.columns.values.tolist()
base_features.remove('target')

# Plot new feature
feat_name = 'ca'
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_test, model_features=base_features, feature=feat_name)

# Show plot
pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
feat_name = 'age'
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_test, model_features=base_features, feature=feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
feat_name = 'oldpeak'
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_test, model_features=base_features, feature=feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
# Create explainer
explainer = shap.TreeExplainer(model)

# Get SHAP_values
shap_values = explainer.shap_values(X_test)

# Plot SHAP_Valuesfeat
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
# Use summary plot module
shap.summary_plot(shap_values[1], X_test)

In [ ]:
def heart_disease_risk_factors(model, patient):
    # Initialize our explainer with our model
    explainer = shap.TreeExplainer(model)
    
    # Put in the characteristics for our patient
    shap_values = explainer.shap_values(patient)
    
    # Create shap
    shap.initjs()
    return shap.force_plot(explainer.expected_value[1], shap_values[1], patient)

In [ ]:
# The data
data_for_prediction = X_test.iloc[1,:].astype(float)

# Call our SHAP 
heart_disease_risk_factors(model, data_for_prediction)

In [ ]:
# Create our data
data_for_prediction = X_test.iloc[3,:].astype(float)

# Predict our data
heart_disease_risk_factors(model, data_for_prediction)

In [ ]:
X_test.head()